## Groq API

In [1]:
from dotenv import load_dotenv
import os
import getpass


# Load environment variables from .env file
load_dotenv()

# Access groq_key
groq_key = os.getenv("GROQ_API_KEY")
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass(groq_key)

## Loading, Splitting, Storing PDFs

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

# === PDF Files ===
pdf_paths = [
    "PDFs/Erickson_Kretschmer_Mendis_chapter_4_PD.pdf",
    "PDFs/Vox-Jenkins.pdf",  # Public domain + culture
    "PDFs/Public Domain and Access to Knowledge.pdf",  # DigitalCommons UGA
    "PDFs/Giblin - What Happens When Books Enter the Public Domain.pdf"  # Harvard Ruggie
]

# Load all PDFs
pdf_docs = []
for path in pdf_paths:
    loader = PyMuPDFLoader(path)
    pdf_docs.extend(loader.load())


all_docs = pdf_docs

print(f"✅ Loaded {len(all_docs)} documents from {len(pdf_paths)} PDFs")


✅ Loaded 131 documents from 4 PDFs


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configure the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # characters per chunk
    chunk_overlap=100    # overlap to preserve context
)

# Split all docs
split_docs = text_splitter.split_documents(all_docs)

print(f"Total chunks created: {len(split_docs)}")


Total chunks created: 992


In [4]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import os

# 1. Set up the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Create FAISS vector store
vectorstore = FAISS.from_documents(split_docs, embedding_model)

## LLM

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
from langchain.chains import RetrievalQA

# Create the retriever from your vectorstore
retriever = vectorstore.as_retriever()

# Create the RAG chain (retrieval-augmented generation)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # standard prompt-stuffing approach
    retriever=retriever
)

# Test it
query = "What is public domain according to Jenkins?"
response = rag_chain.run(query)
print(response)


C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_14188\3080653135.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)


According to Jennifer Jenkins, the public domain is the realm of material that is not covered by intellectual property rights and is therefore free for everyone to use and to build upon.


In [14]:
query = "What is the legal framework discussed in Chapter 4 on the public domain?"
response = rag_chain.run(query)
print(response)


Unfortunately, I don't have information on Chapter 4 of the text you provided. The text only mentions that the chapter will proceed by providing an overview of current ambiguity in the boundaries of the public domain in recent UK and selected international case law, as well as resulting challenges raised for potential users.


## Multi-User (User ID Handling) First Approach that didn't work well with retrieving tool

In [7]:
from langchain.agents import Tool
from langchain.tools.retriever import create_retriever_tool

retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="document_search",
    description="Use this tool to search information from uploaded documents"
)


In [30]:
from langchain.tools import tool

@tool
def document_search(query: str) -> str:
    """
    Use this tool to search information from uploaded documents.
    """
    results = retriever.invoke(query)
    return str(results)

In [34]:
def document_search(query: str) -> str:
    results = retriever.invoke(query)
    return str(results)

# ✅ Wrap in Tool
retriever_tool = Tool(
    name="document_search",
    func=document_search,
    description="Use this tool to search information from uploaded documents."
)

In [35]:
from langgraph.graph import Graph
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# 1. Define tools and agent
tools = [retriever_tool]
memory = MemorySaver()
agent_node = create_react_agent(llm, tools, checkpointer=memory)

# 2. Build graph
graph = Graph()
graph.add_node("agent", agent_node)
graph.set_entry_point("agent")
graph.set_finish_point("agent")  # ✅ tells LangGraph to return output from this node



In [38]:
# ✅ Compile your graph
app = graph.compile()

def chat_with_docs(message, thread_id, user_id):
    config = {"configurable": {"thread_id": thread_id}}
    input_message = {"role": "user", "content": message}
    output = app.invoke({"messages": [input_message]}, config)
    print(output)
    return output["messages"][-1].content
    # # ✅ Use app.stream() instead of graph.stream()
    # for step in app.stream(
    #     {"messages": [input_message]},
    #     config,
    #     stream_mode="values"
    # ):
    #     step["messages"][-1].pretty_print()
    #     new_content = step["messages"][-1].content

    # return new_content


In [39]:
thread_id = "abc1234"
user_id = "678"
# message = "What is the legal framework discussed in Chapter 4 on the public domain?"
# message = "Hi, I'm Youssef!"
message = "What is the legal framework discussed in Chapter 4 on the public domain?"
final_response = chat_with_docs(message, thread_id, user_id)
print(final_response)

{'messages': [HumanMessage(content='What is the legal framework discussed in Chapter 4 on the public domain?', additional_kwargs={}, response_metadata={}, id='9ba31e9e-d973-467e-bca9-5fd4bb08edc3'), AIMessage(content='<document_search>Legal framework discussed in Chapter 4 on the public domain</document_search>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 243, 'total_tokens': 263, 'completion_time': 0.035954456, 'prompt_time': 0.014453326, 'queue_time': 0.089416197, 'total_time': 0.050407782}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8ab2e50475', 'finish_reason': 'stop', 'logprobs': None}, id='run--735a7d65-a8d4-4819-a1d4-86bcbf2094ad-0', usage_metadata={'input_tokens': 243, 'output_tokens': 20, 'total_tokens': 263})]}
<document_search>Legal framework discussed in Chapter 4 on the public domain</document_search>


## Multi-User (User ID Handling) Second Approach with chat history managed manually

In [ ]:
examples=[
    "What is the legal framework discussed in Chapter 4 on the public domain?",
    "How do the authors define cultural commons?",
    "Why does Vox argue that the public domain is shrinking?",
    "How does the public domain support creativity according to the Vox PDF?",
    "What role does the public domain play in access to knowledge?",
    "How does copyright affect the spread of knowledge?",
    "What are the main effects of books entering the public domain?",
    "How does the public benefit when copyright expires?"
]

In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory

# Global session store
user_sessions = {}

def document_search(query: str) -> str:
    results = retriever.invoke(query)
    return str(results)

retriever_tool = Tool(
    name="document_search",
    func=document_search,
    description="Use this tool to search information from uploaded documents."
)

tools = [retriever_tool]

def get_user_agent(user_id):
    # Check if agent exists for this user
    if user_id not in user_sessions:
        # Initialize new memory and agent for this user
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        agent = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
            memory=memory,
            verbose=True
        )
        user_sessions[user_id] = agent
    return user_sessions[user_id]

# Function to handle chat for multi-users
def chat_with_docs(message, user_id):
    agent = get_user_agent(user_id)
    return agent.run(message)


In [ ]:
# Example usage
user_id = "abc1234"
message = "What is the legal framework discussed in Chapter 4 on the public domain?"
response = chat_with_docs(message, user_id)
print(response)

C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_14188\1612736705.py:24: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_14188\1612736705.py:25: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: document_search
Action Input: Chapter 4 on the public domain
Observation: [Document(id='aa5fbc61-17d6-4dbd-abf9-a118db17877c', metadata={'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20200106170812Z00'00'", 'source': 'PDFs/Erickson_Kretschmer_Mendis_chapter_4_PD.pdf', 'file_path': 'PDFs/Erickson_Kretschmer_Mendis_chapter_4_PD.pdf', 'total_pages': 30, 'format': 'PDF 1.3', 'title': 'Erickson_Kretschmer_Mendis_Elgar_Ch4_Drexl (3)', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20200106170812Z00'00'", 'trapped': '', 'modDate': "D:20200106170812Z00'00'", 'creationDate': "D:20200106170812Z00'00'", 'page': 3}, page_content='The chapter will proceed by providing an overview of current ambiguity in \nthe boundaries of the public domain in recent UK and selected international \ncase law, as well as resulting challenges raised for potential users. Thereafter

In [44]:
import gradio as gr

# ✅ Gradio interface definition
with gr.Blocks() as demo:
    gr.Markdown("### 📚 Multi-user Document QA Chatbot")
    
    with gr.Row():
        user_id_input = gr.Textbox(label="User ID")
    
    with gr.Row():
        message_input = gr.Textbox(label="Your Message")
    
    output = gr.Textbox(label="Agent Response")

    submit_btn = gr.Button("Send")
    
    submit_btn.click(
        fn=chat_with_docs,
        inputs=[message_input, user_id_input],
        outputs=output
    )

# ✅ Launch Gradio app
if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Youssef, it's nice to meet you again. I remember our previous conversation about the public domain. How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Youssef, it's nice to meet you. I'm Assistant, a large language model here to help with any questions or topics you'd like to discuss. How are you today?

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: document_search
Action Input: search for information about public domain and creativity in the Vox PDF
Observation: [Document(id='ef62d023-67d8-4842-9bda-0d12706c257b', metadata={'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20200106170812Z00'00'", 'source': 'PDFs/Erickson_Kretschmer_Mendis_chapter_4_PD.pdf', 'file_path': 'PDFs/Erickson_Kretschmer_Mendis_chapter_4_PD.pdf', 't

## (Redis) Multi-user conversational that stores conversation history for each user separately.

### Redis approach following documentation

In [ ]:
!pip install -qU langchain-redis langchain-openai redis
!pip install langchain_redis


In [9]:
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_redis import RedisChatMessageHistory

In [16]:
# Initialize RedisChatMessageHistory
history = RedisChatMessageHistory(session_id="user_123", redis_url=REDIS_URL)

# Add messages to the history
history.add_user_message("Hello, AI assistant!")
history.add_ai_message("Hello! How can I assist you today?")

# Retrieve messages
print("Chat History:")
for message in history.messages:
    print(f"{type(message).__name__}: {message.content}")

14:07:48 redisvl.index.index INFO   Index already exists, not overwriting.


Chat History:
HumanMessage: Hello, AI assistant!
AIMessage: Hello! How can I assist you today?
HumanMessage: Hello, AI assistant!
AIMessage: Hello! How can I assist you today?


In [17]:
# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)


# Create the conversational chain
chain = prompt | llm


# Function to get or create a RedisChatMessageHistory instance
def get_redis_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(session_id, redis_url=REDIS_URL)


# Create a runnable with message history
chain_with_history = RunnableWithMessageHistory(
    chain, get_redis_history, input_messages_key="input", history_messages_key="history"
)

# Use the chain in a conversation
response1 = chain_with_history.invoke(
    {"input": "Hi, my name is Alice."},
    config={"configurable": {"session_id": "alice_123"}},
)
print("AI Response 1:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What's my name?"}, config={"configurable": {"session_id": "alice_123"}}
)
print("AI Response 2:", response2.content)

14:07:51 redisvl.index.index INFO   Index already exists, not overwriting.


14:07:52 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
AI Response 1: Hello Alice, we've already met. How can I assist you today?
14:07:52 redisvl.index.index INFO   Index already exists, not overwriting.
14:07:52 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
AI Response 2: Your name is Alice.


In [19]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()  # optional if you want LangGraph memory; here you will use Redis

tools = [retriever_tool]

# Create the ReAct agent with the tool
agent = create_react_agent(llm, tools)


In [26]:
from langchain.memory import RedisChatMessageHistory

def get_redis_history(session_id: str):
    return RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)


In [27]:
from langchain_core.runnables.history import RunnableWithMessageHistory

agent_with_history = RunnableWithMessageHistory(
    agent,
    get_redis_history,
    input_messages_key="messages",  # for agent inputs structured as messages
    history_messages_key="history",
)


In [ ]:
# Example usage
input_message = {"role": "user", "content": "What's the legal framework of public domain?"}

response = agent_with_history.invoke(
    {"messages": [input_message]},
    config={"configurable": {"session_id": "user_123"}},
)

print("AI Response:", response["messages"][-1].content)


14:13:39 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
14:13:40 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
14:13:40 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
14:13:40 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
14:13:41 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
14:13:41 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
14:13:41 groq._base_client INFO   Retrying request to /openai/v1/chat/completions in 23.000000 seconds
14:14:05 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
14:14:05 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
14:14:05 gr

### Gradio App with Chat history per each user

In [ ]:
import gradio as gr
import redis
import pickle
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import RedisChatMessageHistory


#  Define retriever tool function
def document_search(query: str) -> str:
    results = retriever.invoke(query)
    return str(results)

retriever_tool = Tool(
    name="document_search",
    func=document_search,
    description="Use this tool to search information from uploaded documents."
)

tools = [retriever_tool]

# Initialize Redis client
redis_client = redis.Redis(host="localhost", port=6379, db=0)

#  Function to get agent with Redis-backed memory
def get_user_agent(user_id):
    chat_history = RedisChatMessageHistory(
        session_id=user_id,
        url="redis://localhost:6379/0"
    )

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        chat_memory=chat_history,
        return_messages=True
    )

    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True
    )
    return agent, chat_history


In [ ]:
def chat_with_docs(message, user_id):
    agent, _ = get_user_agent(user_id)
    response = agent.run(message)
    return response

# Function to retrieve and format history for a user_id
def view_history(user_id):
    _, chat_history = get_user_agent(user_id)
    messages = chat_history.messages

    if not messages:
        return "No history found for this user ID."

    history_str = ""
    for msg in messages:
        role = "User" if msg.type == "human" else "AI"
        history_str += f"{role}: {msg.content}\n\n"

    return history_str

# Gradio interface with two tabs
with gr.Blocks() as demo:
    gr.Markdown("### 📚 Multi-user Document QA Chatbot with Redis Memory and History Viewer")

    with gr.Tabs():
        # 🔷 Tab 1: Chat
        with gr.Tab("Chat"):
            with gr.Row():
                user_id_input = gr.Textbox(label="User ID")

            with gr.Row():
                message_input = gr.Textbox(label="Your Message")

            output = gr.Textbox(label="Agent Response")

            submit_btn = gr.Button("Send")
            submit_btn.click(
                fn=chat_with_docs,
                inputs=[message_input, user_id_input],
                outputs=output
            )

        # 🔷 Tab 2: View History
        with gr.Tab("View History"):
            user_id_history_input = gr.Textbox(label="User ID")
            history_output = gr.Textbox(label="Conversation History", lines=20)

            view_btn = gr.Button("View History")
            view_btn.click(
                fn=view_history,
                inputs=user_id_history_input,
                outputs=history_output
            )

# Launch Gradio app
if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Youssef, it's nice to meet you. I'm Assistant, a large language model here to help with any questions or topics you'd like to discuss. How are you today?

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: When copyright expires, the public benefits in several ways. One of the main benefits is that copyrighted works enter the public domain, which means that they are no longer protected by copyright law. This allows anyone to use, reproduce, and distribute the work without needing to obtain permission or pay royalties.

As a result, the public can access and enjoy these works freely, which can lead to increased creativity, innovation, and cultural enrichment. For example, when a famous novel or movie enters the public domain, it can be freely adapted, remixed, and reinterpreted by others, leading to new and innovative works.

Additionally, the expiration